In [1]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.4 MB/s eta 0:00:00


In [2]:
!pip install pytesseract   #optical character recognition (OCR)

In [3]:
!pip install opencv-python

In [4]:
!!pip install pyngrok

['Collecting pyngrok',
 '  Downloading pyngrok-7.3.0-py3-none-any.whl.metadata (8.1 kB)',
 'Requirement already satisfied: PyYAML>=5.1 in /usr/local/lib/python3.12/dist-packages (from pyngrok) (6.0.2)',
 'Downloading pyngrok-7.3.0-py3-none-any.whl (25 kB)',
 'Installing collected packages: pyngrok',
 'Successfully installed pyngrok-7.3.0']

In [ ]:
!pip install spacy

In [5]:
!pip install skillNer

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.9/355.9 kB 4.8 MB/s eta 0:00:00
  Created wheel for skillNer: filename=skillNer-1.0.3-py3-none-any.whl size=25625 sha256=aba0d5fb3795012608658a452e0669f9922426359e53d0dee61b7e6b6ee60043
  Stored in directory: /root/.cache/pip/wheels/05/61/2f/331f7cbd5980aceaec089ca82ce49c282523d8365bf99e8013
Successfully built skillNer


In [6]:
!pip install google-generativeai

In [7]:
!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
%%writefile jobpost.py
import streamlit as st
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor
import google.generativeai as genai
import json
import os
import re
import subprocess
from PIL import Image, ImageEnhance, ImageFilter

#load pytesseract model
import pytesseract

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_lg")
except OSError:
    st.warning("Downloading spaCy model 'en_core_web_lg'...")
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_lg"])
    nlp = spacy.load("en_core_web_lg")

# Initialize SkillExtractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# Gemini API
api = 'api_key'
genai.configure(api_key=api)

# Preprocess text
def preprocess_text(text):
    clean_text = re.sub(r'[^A-Za-z0-9.,;:()\- ]+', ' ', text)
    clean_text = re.sub(r'\s+', ' ', clean_text)
    return clean_text.strip().lower()

# Gemini skills
def get_skills_from_gemini(job_description):
    prompt = f"Extract only the skills from the following job description:\n{job_description}"
    try:
        model = genai.GenerativeModel("models/gemini-1.5-flash-002")
        response = model.generate_content(prompt)
        skills = response.text.strip().split("\n")
        return [s.strip("*•- ").strip() for s in skills if s.strip()]
    except Exception as e:
        st.error(f"Gemini API error: {str(e)}")
        return []

# SkillNer skills
def extract_skills(job_description):
    extracted_skills = skill_extractor.annotate(job_description)
    return [match['doc_node_value'] for match in extracted_skills['results'].get('soft_matches', [])]

# OCR from image (enhanced)
def extract_text_from_image(image_file):
    image = Image.open(image_file).convert('L')  # Convert to grayscale
    image = image.filter(ImageFilter.SHARPEN)  # Sharpen image
    image = ImageEnhance.Contrast(image).enhance(2)  # Increase contrast
    text = pytesseract.image_to_string(image)
    return text

# Streamlit UI
st.title("📄 AI-Powered Job Post Skill Extractor")

option = st.radio("Choose input method:", ("Text", "Image"))

if option == "Text":
    job_post = st.text_area("Enter the Jop Posting here :")
elif option == "Image":
    uploaded_image = st.file_uploader("Upload job post image (PNG/JPG):", type=["png", "jpg", "jpeg"])
    if uploaded_image:
        st.image(uploaded_image, caption="Uploaded Image", use_container_width=True)
        extracted_text = extract_text_from_image(uploaded_image)
        st.text_area("Extracted Text from Image:", extracted_text, height=200)
        job_post = extracted_text
    else:
        job_post = ""

if st.button("Extract Skills"):
    if job_post.strip():
        cleaned_text = preprocess_text(job_post)
        gemini_skills = get_skills_from_gemini(cleaned_text)
        skillner_skills = extract_skills(cleaned_text)
        final_skills = sorted(set(gemini_skills + skillner_skills))

        st.subheader("✅ Extracted Skills:")
        st.json({"extracted_skills": final_skills})
    else:
        st.warning("Please enter or upload a job description.")


Writing jobpost.py


In [9]:
#setup ngrok with your authonken
from pyngrok import ngrok
!ngrok authtoken <your_authtoken>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
!streamlit run jobpost.py &>/dev/null&

In [11]:
!pgrep -f streamlit

1366


In [12]:
puplic_url = ngrok.connect('8501')
puplic_url

<NgrokTunnel: "https://f2393f013f75.ngrok-free.app" -> "http://localhost:8501">

In [ ]:
ngrok.kill()